In [28]:
%pip install jinja2
%pip install python-dotenv
%pip install requests
%pip install wonderwords

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [29]:
import json
import os
import random
import requests

from dotenv import load_dotenv
from IPython.display import display, Markdown
from wonderwords import RandomWord

In [30]:
# SPOTIFY API CONSTANTS

load_dotenv()

CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
SPOTIFY_BASE_URL = 'https://api.spotify.com/v1/'

In [31]:
# SPOTIFY API AUTHENTICATION

def get_spotify_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    
    response = requests.post(url, headers=headers, data=data)
    response_data = response.json()
    
    if response.status_code == 200:
        return response_data['access_token']
    else:
        raise Exception(f"Error fetching token: {response_data}")

TOKEN = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
HEADERS = {
        "Authorization": f"Bearer {TOKEN}"
    }

In [ ]:
# SPOTIFY API FUNCTIONS

r = RandomWord()
def get_random_album_id():
    random_word = r.word()
    
    search_url = SPOTIFY_BASE_URL + "search"
    params = {
        "q": random_word,
        "type": "album",
        "tag": "hipster", # to get more obscure results
        "limit": 50
    }
    response = requests.get(search_url, headers=HEADERS, params=params)
    response_data = response.json()
    
    albums = response_data.get('albums', {}).get('items', [])
    non_single_albums = [album for album in albums if album['album_type'] == 'album']
    
    if non_single_albums:
        random_album = random.choice(non_single_albums)
        return random_album['id']
    else:
        return None

def get_album_info(album_id):
    url = SPOTIFY_BASE_URL + f"albums/{album_id}"
    response = requests.get(url, headers=HEADERS)
    album = response.json()
    
    album_info = {
        'name': album['name'],
        'id': album['id'],
        'date': album['release_date'],
        'artists': [a['name'] for a in album['artists']],
        'cover': album['images'][0]['url'],
        'genres': album['genres'], # most often not yet classified
        'label': album['label']
    }

    return album_info

def get_album_tracks(album_id):
    url = SPOTIFY_BASE_URL + f"albums/{album_id}/tracks"
    response = requests.get(url, headers=HEADERS)
    response_data = response.json()

    tracks = []
    for t in response_data.get('items', []):
        track = {
            'name': t['name'],
            'artists': [a['name'] for a in t['artists']]
        }
        tracks.append(track)

    return tracks

def get_random_album_information():
    album_id = get_random_album_id()
    album = get_album_info(album_id)
    album['tracks'] = get_album_tracks(album_id)
    return album

def display_album(album):
    if album:       
        display(Markdown(f"**{album['name']}**"))
        display(Markdown(f"![Album Cover]({album['cover']})"))
        display(Markdown(f"**Artists**: {', '.join(album['artists'])}"))
        display(Markdown(f"**Release Date**: {album['date']}"))
        display(Markdown(f"**Genres**: {', '.join(album['genres'])}"))
        display(Markdown(f"**Label**: {album['label']}"))
        display(Markdown(f"**Tracks**:"))
        for track in album['tracks']:
            display(Markdown(f"{track['name']} - {', '.join(track['artists'])}"))
    else:
        display(Markdown("No albums found. Try again!"))

In [ ]:
# SAVE ALBUM INFORMATION TO JSON FILE

NUMBER_OF_ALBUMS = 2
for i in range(NUMBER_OF_ALBUMS):
    album = get_random_album_information()

    # display_album(album)

    # Ensure the directory exists
    os.makedirs('input/albums', exist_ok=True)

    with open(f"input/albums/{album['id']}.json", 'w') as f:
        json.dump(album, f, indent=4)